In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import json
import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.item.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Json_Compatibility'] != '{}'].reset_index(drop=True)

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import json
        import requests

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        try:
            df_temp, json_, page = pd.DataFrame(), json.loads(input_.loc[a, 'Json_Compatibility']), 1
            while page <= json_['page']:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.post(f'''{json_['api']}?module_groups=PART_FINDER&referrer=VIEWITEM&offset={page*20-20}&module=COMPATIBILITY_TABLE''',
                                             data=json.dumps(json_['data']),
                                             headers=get_header(ua=False, **{'Content-Type': 'application/json; charset=utf-8'}),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))


                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_header = [cell['textSpans'][0]['text'].strip() for cell in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']]
            
                # = = = = = = = = = = = = = = =
            
                list_row = [[cell['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in cell else cell['textSpans'][0]['text'].strip() for cell in row['cells']] for row in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]
            
                # = = = = = = = = = = = = = = =
            
                df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                             'Item Number': input_.loc[a, 'Item Number'],
                                             'Json_Compatibility': input_.loc[a, 'Json_Compatibility'],
                                             'Page': page,
                                             'Row': [i+1 for i in range(len(list_row))]})
            
                for i, row in zip(range(len(list_row)), list_row):
                    for j, header in zip(range(len(list_header)), list_header):
                        df_temp_temp.loc[i, header] = row[j]
            
                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
                
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
            
                # = = = = = = = = = = = = = = =
            
                page += 1

            # = = = = = = = = = = = = = = =

            if df_temp.empty:
                raise
                            
            # = = = = = = = = = = = = = = =

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Item Number': input_.loc[a, 'Item Number'],
                                     'Json_Compatibility': input_.loc[a, 'Json_Compatibility']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.vehicle-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：4

[尝试次数：52] - 3.113134507797 > Page 1
[剩余数量：0] - [当前时间：17:40:52]

[尝试次数：54] - 5.115186863085 > Page 1
[剩余数量：0] - [当前时间：17:40:53]

[尝试次数：49] - 1.110940194723 > Page 1
[剩余数量：0] - [当前时间：17:40:53]

[尝试次数：51] - 6.115186863204 > Page 1
[剩余数量：0] - [当前时间：17:40:54]

[尝试次数：1] - 5.115186863085 > Page 2
[剩余数量：0] - [当前时间：17:40:55]

[尝试次数：1] - 1.110940194723 > Page 2
[剩余数量：0] - [当前时间：17:40:56]

[尝试次数：1] - 5.115186863085 > Page 3
[剩余数量：0] - [当前时间：17:40:56]

[尝试次数：1] - 6.115186863204 > Page 2
[剩余数量：0] - [当前时间：17:40:57]

[尝试次数：1] - 1.110940194723 > Page 3
[剩余数量：0] - [当前时间：17:40:57]

[尝试次数：1] - 5.115186863085 > Page 4
[剩余数量：0] - [当前时间：17:40:58]

[尝试次数：1] - 6.115186863204 > Page 3
[剩余数量：0] - [当前时间：17:40:58]

[尝试次数：1] - 1.110940194723 > Page 4
[剩余数量：0] - [当前时间：17:40:59]

[尝试次数：1] - 5.115186863085 > Page 5
[剩余数量：0] - [当前时间：17:40:59]

[尝试次数：1] - 1.110940194723 > Page 5
[剩余数量：0] - [当前时间：17:41:00]

[尝试次数：1] - 5.115186863085 > Page 6
[剩余数量：0] - [当前时间：17:41:01]

[尝试次数：1] - 1.110940194723 > Page 6
[剩余数量：0] 